In [9]:
import urllib
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime

def chkHoliday(year,month,day):
    holiday_url = "https://api.manana.kr/calendar.xml?year="+str(year)+"&month="+str(month).zfill(2)+"&day="+str(day)+"&category=all&nation=kr"
    hd_opn = urllib.request.urlopen(holiday_url)
    hd_red = hd_opn.read()
    soup3 = BeautifulSoup(hd_red,"html.parser")
    if soup3.find('category') == None:
        return 'F'
    elif soup3.find('category').text == 'holiday' : 
        return 'T'

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

def getrain(year):
    rain_url = "http://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy="+str(year)+"&obs=21"
    rain_opn = urllib.request.urlopen(rain_url)
    rain_red = rain_opn.read()
    soup2 = BeautifulSoup(rain_red,"html.parser")
    df_weather= pd.DataFrame()
    table = soup2.find('table', attrs={'class': 'table_develop'})
    tbody = table.find('tbody')
    trs = tbody.find_all('tr')

    for i in range(31):
        tds = trs[i].find_all('td')
        for j in range(1,13):
            if tds[j].text == '\xa0':
                df_weather.set_value('{0}'.format(str(i+1).zfill(2)),'{0}'.format(str(j).zfill(2)),"NaN")
                continue
            df_weather.set_value('{0}'.format(str(i+1).zfill(2)),'{0}'.format(str(j).zfill(2)),tds[j].text)
    return df_weather

def gettemperature(year):
    tem_url = "http://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy="+str(year)+"&obs=07"
    tem_opn = urllib.request.urlopen(tem_url)
    tem_red = tem_opn.read()
    soup2 = BeautifulSoup(tem_red,"html.parser")
    df_weather= pd.DataFrame()
    table = soup2.find('table', attrs={'class': 'table_develop'})
    tbody = table.find('tbody')
    trs = tbody.find_all('tr')

    for i in range(31):
        tds = trs[i].find_all('td')
        for j in range(1,13):
            if tds[j].text == '\xa0':
                df_weather.set_value('{0}'.format(str(i+1).zfill(2)),'{0}'.format(str(j).zfill(2)),"0")
                continue
            df_weather.set_value('{0}'.format(str(i+1).zfill(2)),'{0}'.format(str(j).zfill(2)),tds[j].text)
    return df_weather

In [ ]:
df_daybox = pd.read_csv('daily boxoffice7_9(C2).csv')

In [ ]:
df_db79 = p.read_csv('daybox(7-9).csv')

In [3]:
df_daybox.head()

,Unnamed: 0,date,day,movieNm,profit,profitPerc,profitAcc,audiCnt,audiAcc,scrnCnt,showCnt
0,0,2012-07-01,sun,어메이징 스파이더맨,4595645000,0.723,1.523312e+10,509439,1701489,1107,5810
1,1,2012-07-01,sun,마다가스카3 : 이번엔 서커스다!,395133000,0.062,1.125970e+10,54273,1446322,398,990
2,2,2012-07-01,sun,내 아내의 모든 것,274991000,0.043,3.279340e+10,35473,4395678,237,787
3,3,2012-07-01,sun,후궁 : 제왕의 첩,273896500,0.043,1.808728e+10,35447,2458905,316,1216
4,4,2012-07-01,sun,캐빈 인 더 우즈,259264000,0.041,9.979225e+08,32991,131502,284,1089


In [10]:
df_daybox['rainfall'] = ''
df_daybox['temperature'] =''
df_daybox['holiday'] = ''
idx=0
for g in df_daybox['date']:
    y = df_daybox['date'][idx][0:4]# 2012
    df_rain=getrain(y)
    df_temper=gettemperature(y)
    print("------------------{0}------------------".format(y))
    m = df_daybox['date'][idx][5:7] #2012
    print("------------------{0}------------------".format(m))
    for dcnt in range(10*last_day_of_month(datetime.date(int(y), int(m), 4)).day):
        d = df_daybox['date'][idx][8:10]
        print("------------------{0}------------------".format(d))
        df_daybox.set_value(idx,'rainfall',df_rain[m][d])
        df_daybox.set_value(idx,'temperature',df_temper[m][d])
        df_daybox.set_value(idx,'holiday',str(chkHoliday(y,m,d)))
        print(str(idx)+" - "+df_rain[m][d])
        print(str(idx)+" - "+df_temper[m][d])
        idx=idx+1
        sleep(0.1)


C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import k

------------------2012------------------
------------------07------------------
------------------01------------------


C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


0 - 0.0
0 - 21.6
------------------01------------------
1 - 0.0
1 - 21.6
------------------01------------------
2 - 0.0
2 - 21.6
------------------01------------------
3 - 0.0
3 - 21.6
------------------01------------------
4 - 0.0
4 - 21.6
------------------01------------------
5 - 0.0
5 - 21.6
------------------01------------------
6 - 0.0
6 - 21.6
------------------01------------------
7 - 0.0
7 - 21.6
------------------01------------------
8 - 0.0
8 - 21.6
------------------01------------------
9 - 0.0
9 - 21.6
------------------02------------------
10 - NaN
10 - 24.0
------------------02------------------
11 - NaN
11 - 24.0
------------------02------------------
12 - NaN
12 - 24.0
------------------02------------------
13 - NaN
13 - 24.0
------------------02------------------
14 - NaN
14 - 24.0
------------------02------------------
15 - NaN
15 - 24.0
------------------02------------------
16 - NaN
16 - 24.0
------------------02------------------
17 - NaN
17 - 24.0
---------------

KeyboardInterrupt: 

In [6]:
len(df_daybox['date'])

5820

In [89]:
y = box_dt[12][0:4]
gweather=getweather(y)
m = df_daybox['date'][12][5:7]
d = df_daybox['date'][12][8:10]
hi[m][d]

'51.5'

In [17]:
hi=getweather(2012)
hi

C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Jin1ib\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,01,02,03,04,05,06,07,08,09,10,11,12
01,0.0,0,0,0,0,0,0.0,0,0,0,0,0
02,0,0,0.3,22.0,0.4,0,0,0,0,0,0,0
03,0.4,0.1,0,35.0,0.2,0,25.0,0,0,0,0,5.4
04,0.0,0.2,0.0,0,0,0,0.3,0,55.5,0,4.0,0
05,0,0,7.0,0,0,0.0,72.0,0,10.5,0,8.0,5.8
06,0,0.2,8.5,0,0,0,102.5,0,0,0,5.0,0.1
07,0,0.0,0,0,0,0,0,0,3.0,0,0,1.9
08,0,0,0,0,0,2.0,0,0,25.0,0,0,0
09,0.0,0,0,0,0,0,0,0,0,0,0,0
10,0,0.0,0,10.5,0,0,6.5,0.3,0,13.0,0,0


In [8]:
df_daybox[df_daybox['date'] == '2014-08-31']

,Unnamed: 0,date,day,movieNm,profit,profitPerc,profitAcc,audiCnt,audiAcc,scrnCnt,showCnt,rainfall,temperature
2450,0,2014-08-31,sun,해적: 바다로 간 산적,1859677700,0.258,5.391416e+10,237986,7042030,681,3067,NaN,24.1
2451,1,2014-08-31,sun,인투 더 스톰,1866898500,0.259,5.577446e+09,236345,713799,585,3019,NaN,24.1
2452,2,2014-08-31,sun,명량,1019603400,0.142,1.307520e+11,129731,16949760,495,2130,NaN,24.1
2453,3,2014-08-31,sun,비긴 어게인,862589400,0.120,6.441772e+09,104453,813744,375,1478,NaN,24.1
2454,4,2014-08-31,sun,닌자터틀,830658000,0.115,2.630485e+09,98765,313781,460,2298,NaN,24.1
2455,5,2014-08-31,sun,안녕 헤이즐,222598500,0.031,5.623374e+09,27671,721675,258,658,NaN,24.1
2456,6,2014-08-31,sun,브릭 맨션: 통제불능 범죄구역,88445000,0.012,4.785327e+08,11376,66054,274,687,NaN,24.1
2457,7,2014-08-31,sun,해무(海霧,84171000,0.012,1.160334e+10,10219,1461574,224,461,NaN,24.1
2458,8,2014-08-31,sun,비행기 2: 소방구조대,59753400,0.008,2.238070e+09,8085,310008,125,207,NaN,24.1
2459,9,2014-08-31,sun,더 퍼지 : 거리의 반란,65400100,0.009,3.818884e+08,7925,50280,190,412,NaN,24.1


In [11]:
df_daybox.to_excel('daybox(7-9).xlsx',sheet_name='Sheet1')

In [31]:
import urllib
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime

In [32]:
def chkHoliday(year,month,day):
    holiday_url = "https://api.manana.kr/calendar.xml?year="+str(year)+"&month="+str(month).zfill(2)+"&day="+str(day)+"&category=all&nation=kr"
    token = 'f51eb3c5-4787-40e1-a7c2-011f228701ef'
    request = urllib.request.Request(holiday_url)
    request.add_header(token)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    hd_opn = urllib.request.urlopen(holiday_url)
    hd_red = hd_opn.read()
    soup3 = BeautifulSoup(hd_red,"html.parser")
    if soup3.find('category') == None:
        return 'F'
    elif soup3.find('category').text == 'holiday' : 
        return 'T'


In [27]:
df_db79 = pd.read_csv('daybox(7-9).csv')

In [20]:
df_db79.head()

,Unnamed: 0,Unnamed: 0.1,date,day,movieNm,profit,profitPerc,profitAcc,audiCnt,audiAcc,scrnCnt,showCnt,rainfall,temperature
0,0,0,2012-07-01,sun,어메이징 스파이더맨,4595645000,0.723,1.523312e+10,509439,1701489,1107,5810,0.0,21.6
1,1,1,2012-07-01,sun,마다가스카3 : 이번엔 서커스다!,395133000,0.062,1.125970e+10,54273,1446322,398,990,0.0,21.6
2,2,2,2012-07-01,sun,내 아내의 모든 것,274991000,0.043,3.279340e+10,35473,4395678,237,787,0.0,21.6
3,3,3,2012-07-01,sun,후궁 : 제왕의 첩,273896500,0.043,1.808728e+10,35447,2458905,316,1216,0.0,21.6
4,4,4,2012-07-01,sun,캐빈 인 더 우즈,259264000,0.041,9.979225e+08,32991,131502,284,1089,0.0,21.6


In [28]:
df_daybox['holiday'] = ''
idx=0
for g in df_daybox['date']:
    y = df_daybox['date'][idx][0:4]# 2012
    print("------------------{0}------------------".format(y))
    m = df_daybox['date'][idx][5:7] #2012
    print("------------------{0}------------------".format(m))
    for dcnt in range(10*last_day_of_month(datetime.date(int(y), int(m), 4)).day):
        d = df_daybox['date'][idx][8:10]
        print("------------------{0}------------------".format(d))
        df_daybox.set_value(idx,'holiday',str(chkHoliday(y,m,d)))
        print(str(idx)+" - "+str(chkHoliday(y,m,d)))
        idx=idx+1


------------------2012------------------
------------------07------------------
------------------01------------------


TypeError: add_header() missing 1 required positional argument: 'val'

In [33]:
chkHoliday(2014,str(2).zfill(2),14)

TypeError: add_header() missing 1 required positional argument: 'val'